In [58]:
import pandas as pd

# Load the CSV file
file_path_q = '../../data/question_survey/question_dataset.csv'
question_data = pd.read_csv(file_path_q)

chatbot_questions = question_data[['question_id_q', 'german_question_text_q', 'english_question_text_q']]
# Save the new Dataframe to a CSV file
chatbot_questions.to_csv('../../data/chatbot_data/chatbot_questions.csv', index=False, quoting=1)
chatbot_questions.to_csv('../../chatbot/data_eval/chatbot_questions.csv', index=False, quoting=1)


In [ ]:
# #COPY FROM chatbot/data_eval/chatbot_answers_test.py and executed in the chatbot environment

# import sys

# # print(f"Python version: {sys.path}")
# sys.path.append("./")

# import pandas as pd
# import json
# from src.chatbot.agents.agent_openai_tools import CampusManagementOpenAIToolsAgent
# from src.chatbot.tools.search_web_tool import visited_links
# from src.config.core_config import settings
# from typing import List
# from datetime import datetime
# # Ignore the warning in on_llm_new_token
# import logging
# class SuppressSpecificWarning(logging.Filter):
#     def filter(self, record):
#         return "unsupported operand type(s) for -" not in record.getMessage()


# logger = logging.getLogger("langchain_core.callbacks.manager")
# logger.addFilter(SuppressSpecificWarning())
# ####################################
# # Exceptions for the agent
# class AgentException(Exception):
#     """Custom exception for agent-related issues."""
#     pass

# # Function to compare memory states
# def assert_memory_states(agent_executor, initial_memory):
#     """
#     Checks if the agent's memory after .clear() matches the initial memory state.
#     """
#     if agent_executor.memory != initial_memory:
#         raise AgentException(
#             f"Memory mismatch for agent {id(agent_executor)}: "
#             f"Expected {initial_memory}, but got {agent_executor.memory}."
#         )
    
# # Function to check if visited links are an empty list
# def assert_visited_links_empty():
#     """
#     Checks if the visited links list is empty.
#     """
#     if visited_links():
#         raise AgentException(
#             "Visited links list is not empty after clearing."
#         )
    
# # check if the language is set to English or German
# def check_language(agent_executor, language):
#     """
#     Checks if the agent's language is set to the correct language.
#     """
#     if agent_executor.language != language:
#         raise AgentException(
#             f"Language mismatch for agent {id(agent_executor)}: "
#             f"Expected {language}, but got {agent_executor.language}."
#         )
# # Function to check for overlaps between queries and DataFrame questions
# def assert_no_overlap_with_df(queries: List[str], df: pd.DataFrame, question_column: str):
#     """
#     Ensures that no query in the provided list is present in the specified column of the DataFrame.

#     Parameters:
#     queries (List[str]): The list of queries to check.
#     df (pd.DataFrame): The DataFrame containing questions.
#     question_column (str): The column name in the DataFrame to check against.

#     Raises:
#     AgentException: If any overlap is found.
#     """
#     overlapping_queries = set(queries).intersection(df[question_column].dropna())
#     if overlapping_queries:
#         raise AgentException(
#             f"Overlapping queries detected in {question_column}: {overlapping_queries}"
#         )

# ####################################

# # Create a fresh German agent
# settings.language = "Deutsch"
# agent_executor_de = CampusManagementOpenAIToolsAgent.run(language="Deutsch")
# initial_memory_de = agent_executor_de.memory.copy()  # Save the initial memory state
# print(visited_links())  # Save the initial visited links

# # Create a fresh English agent
# settings.language = "English"
# agent_executor_en = CampusManagementOpenAIToolsAgent.run(language="English")
# initial_memory_en = agent_executor_en.memory.copy()  # Save the initial memory state

# # Check if the two agents are the same
# if id(agent_executor_de) == id(agent_executor_en):
#     raise AgentException("The two agents share the same memory address, which is not expected!")

# warm_up_queries_en = [
#     "Hello! How can I get started with my application?",
#     "How do I reset my university account password?",
#     "How do I apply for a scholarship?",
#     "How do I apply for a student visa?",
#     "How do I apply for a student loan?",
#     "How do I apply for a student housing?",
# ]

# warm_up_queries_de =[
#     "Hallo! Wie kann ich mit meiner Bewerbung beginnen?",
#     "Wie setze ich mein Universitätskontopasswort zurück?",
#     "Wie bewerbe ich mich um ein Stipendium?",
#     "Wie bewerbe ich mich um ein Studentenvisum?",
#     "Wie bewerbe ich mich um ein Studentendarlehen?",
#     "Wie bewerbe ich mich um eine Studentenwohnung?",
# ]


# def warm_up_agent(warm_up_queries: List[str], agent_executor) -> None:
#     """
#     Downloads models and  prepare the agent for actual tasks.

#     Parameters:
#     warm_up_queries (List[str]): A list of queries to be executed during the warm-up phase.

#     Returns:
#     None
#     """
#     # Warm-up phase
#     for q in warm_up_queries:
#         agent_executor(q)

# # Warm-up phase
# warm_up_agent(warm_up_queries_de, agent_executor_de)
# warm_up_agent(warm_up_queries_en, agent_executor_en)


# # Clear memory and verify states
# print("Clearing memory and links...")
# agent_executor_de.memory.clear()
# agent_executor_en.memory.clear()
# visited_links.clear()

# print("Verifying memory states and links...")
# assert_memory_states(agent_executor_de, initial_memory_de)
# assert_memory_states(agent_executor_en, initial_memory_en)
# assert_visited_links_empty()

# print("All checks passed successfully.")

# # Read the CSV
# df = pd.read_csv("data_eval/chatbot_questions.csv")
# df_copy = df.copy()
# #df_copy = df.head(1).copy()  # for testing

# all_results = []

# # Check for overlaps in German and English queries
# # Could interfere with the context retrieval
# assert_no_overlap_with_df(warm_up_queries_de, df_copy, "german_question_text_q")
# assert_no_overlap_with_df(warm_up_queries_en, df_copy, "english_question_text_q")

# # save the start time for context retrieval
# start_time = datetime.now()

# for i, row in df_copy.iterrows():
#     # ------------------------------------------------------------
#     # 1) GET GERMAN QUESTION
#     # ------------------------------------------------------------
#     question_de = row["german_question_text_q"]
#     question_id = row["question_id_q"]
    
#     # Clear the memory
#     agent_executor_de.memory.clear()
#     assert_memory_states(agent_executor_de, initial_memory_de)
#     # Chatbot returns an answer in German
#     check_language(agent_executor_de, "Deutsch")
#     result_de = agent_executor_de(question_de)
#     visited_links_de = visited_links()
#     # clear visited links
#     visited_links.clear()
#     assert_visited_links_empty()

#     # ------------------------------------------------------------
#     # 2) GET ENGLISH QUESTION
#     # ------------------------------------------------------------
#     question_en = row["english_question_text_q"]

#     # Clear the memory
#     agent_executor_en.memory.clear()
#     assert_memory_states(agent_executor_en, initial_memory_en)
#     # Chatbot should now return an answer in English
#     check_language(agent_executor_en, "English")
#     result_en = agent_executor_en(question_en)
#     visited_links_en = visited_links()
#     # clear visited links
#     visited_links.clear()
#     assert_visited_links_empty()

#     # ------------------------------------------------------------
#     # 3) Combine results
#     # ------------------------------------------------------------
#     row_dict = {
#         "question_id": question_id,
#         "question_de": question_de,
#         "output_de": result_de["output"],
#         "visited_links_function_de": visited_links_de,
#         "question_en": question_en,
#         "output_en": result_en["output"],
#         "visited_links_function_en": visited_links_en,
#         "earliest_start_time": start_time,
#     }
#     all_results.append(row_dict)
    
# # ------------------------------------------------------------
# # 4) SAVE RESULTS
# # ------------------------------------------------------------
# # with open("data_eval/chatbot_responses_test.json", "w", encoding="utf-8") as f:
# #     json.dump(all_results, f, indent=2, ensure_ascii=False)

# df_copy["chatbot_answer_de"] = [res["output_de"] for res in all_results]
# df_copy["chatbot_answer_en"] = [res["output_en"] for res in all_results]
# df_copy["chatbot_links_function_de"] = [
#     ";".join(res["visited_links_function_de"]) for res in all_results
# ]
# df_copy["chatbot_links_function_en"] = [
#     ";".join(res["visited_links_function_en"]) for res in all_results
# ]
# df_copy["earliest_start_time"] = start_time

# df_copy.to_csv(
#     "data_eval/chatbot_answers.csv", index=False, quoting=1
# )

### Add context from chatobot answer, the search query used for the website search and the latency of the whole process

In [60]:
import os
from langsmith import Client
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime, timedelta
import ast
import tqdm

# Load the CSV file
chatbot_answer_data = pd.read_csv('../../chatbot/data_eval/chatbot_answers.csv')

# Convert the earliest_start_time string to a datetime object
chatbot_answer_start_str = chatbot_answer_data["earliest_start_time"][0]
chatbot_answer_start = datetime.strptime(chatbot_answer_start_str, "%Y-%m-%d %H:%M:%S.%f") 

# Load environment variables (e.g., LANGCHAIN_API_KEY, etc.)
load_dotenv()

# Connect to LangSmith
api_key = os.getenv("LANGCHAIN_API_KEY")
project_name = os.getenv("LANGCHAIN_PROJECT")
endpoint = os.getenv("LANGCHAIN_ENDPOINT")
client = Client(api_key=api_key)

# Fetch all root runs that occurred after 'chatbot_answer_start'
root_runs = client.list_runs(
    project_name=project_name,
    start_time=chatbot_answer_start,
    is_root=True,
)

# Prepare a list to store results of each root run
root_run_data = []

# Iterate over all root runs with tqdm for a progress bar
for root_run in tqdm.tqdm(root_runs):
    # Calculate latency if end_time is available
    latency = (root_run.end_time - root_run.start_time).total_seconds() if root_run.end_time else None

    # Initialize variables for storing info about this root run
    tool_run_used = "no search tool used"
    chatbot_context_root = ""
    chatbot_query = ""
    tool_run_id = ""

    # Get child runs triggered by this root run
    child_run_ids = root_run.child_run_ids

    # From those child runs, fetch only the tool runs
    tool_runs = client.list_runs(run_ids=child_run_ids, run_type="tool")

    # Filter only the tool runs whose name is "custom_university_web_search"
    custom_search_runs = [tr for tr in tool_runs if tr.name == "custom_university_web_search"]

    # If there are multiple "custom_university_web_search" runs, sort by start_time descending
    if custom_search_runs:
        # Sort them and pick the latest run
        custom_search_runs_sorted = sorted(custom_search_runs, key=lambda r: r.start_time, reverse=True)
        latest_run = custom_search_runs_sorted[0]

        # Now we know we used the search tool
        tool_run_used = "search tool used"
        tool_run_inputs_str = latest_run.inputs["input"]
        tool_run_inputs_dict = ast.literal_eval(tool_run_inputs_str)

        # Extract the user query from the tool run's inputs
        chatbot_query = tool_run_inputs_dict.get("query", "")
        tool_run_id = str(latest_run.id)

        # If the tool run has child runs, we check if it used summarization (e.g. MapReduceDocumentsChain)
        if latest_run.child_run_ids:
            tool_run_used = "search tool with summarization used"
            tool_run_childs = client.list_runs(run_ids=latest_run.child_run_ids)
            chatbot_context_root = latest_run.outputs["output"]
            # Sort them and pick the earliest run
            tool_run_childs_sorted = sorted(tool_run_childs, key=lambda r: r.start_time)
            # Look for the summarization chain among the child runs
            for child in tool_run_childs_sorted:
                if child.name == "MapReduceDocumentsChain":
                    # Summarization inputs = raw documents
                    tool_summarization_inputs = child.inputs["input_documents"]
                    # Collect all page_content from each doc
                    for doc in tool_summarization_inputs:
                        if isinstance(doc, dict) and "page_content" in doc:
                            chatbot_context_root += doc["page_content"] + "\n"
        else:
            # If no child runs, we can use the output from the tool directly
            chatbot_context_root = latest_run.outputs["output"]

    # Store all data related to this root run in a dictionary
    root_run_data.append({
        "root_run_id": str(root_run.id),
        "chatbot_question": root_run.inputs["input"],
        "start_time": root_run.start_time,
        "end_time": root_run.end_time,
        "latency": latency,
        "chatbot_context": chatbot_context_root,
        "chatbot_query": chatbot_query,
        "tool_run_id": tool_run_id,
        "tool_run_used": tool_run_used
    })

# Convert the list of dictionaries to a DataFrame
df_root_runs = pd.DataFrame(root_run_data)

# Display or save results
print(f"Number of root runs processed: {len(df_root_runs)}")
# Save to a new CSV file (update path/filename as you like)
df_root_runs.to_csv("../../data/chatbot_data/chatbot_meta_data.csv", index=False, quoting=1)
df_root_runs.head()


0it [00:00, ?it/s]

66it [01:05,  1.00it/s]

Number of root runs processed: 66


,root_run_id,chatbot_question,start_time,end_time,latency,chatbot_context,chatbot_query,tool_run_id,tool_run_used
0,184ad8c3-299a-4693-849a-b4bc11ccfb3a,When and how does the registration process tak...,2025-01-09 13:51:58.654014,2025-01-09 13:52:09.735063,11.081049,Information taken from: https://www.uni-osnabr...,Immatrikulation Universität Osnabrück 2024,a0f066ed-de85-4066-a99f-ce1aab5765fb,search tool used
1,492586fd-42f9-42de-83d9-246a5397c851,Wann und wie läuft das Anmeldeverfahren ab?,2025-01-09 13:51:45.732033,2025-01-09 13:51:58.639936,12.907903,Information taken from: https://www.uni-osnabr...,Anmeldeverfahren Universität Osnabrück,ff95caf7-d083-4bbe-a085-078ccbab2359,search tool used
2,9da69d69-4372-42e8-ab4b-866e19c70862,"I work as a working student, can I work as a t...",2025-01-09 13:51:35.875701,2025-01-09 13:51:45.712414,9.836713,Information taken from: https://www.uni-osnabr...,Kann ich als Werkstudent auf Minijob-Basis als...,fde7448b-a90b-4b09-acdd-36ed13909497,search tool used
3,b54cc954-05bd-4256-98d4-fb57a232a292,"Ich arbeite als Werkstudent, kann ich als Tuto...",2025-01-09 13:51:33.400742,2025-01-09 13:51:35.860340,2.459598,,,,no search tool used
4,3bbaf512-f56a-401f-b04e-f073d25e5278,What are the semester dates?,2025-01-09 13:51:25.586970,2025-01-09 13:51:33.376646,7.789676,Information taken from: https://www.uni-osnabr...,Semestertermine Universität Osnabrück,14ee3954-f40a-4213-ae60-4eb36ea4af2c,search tool used


In [69]:
import pandas as pd

def merge_chatbot_answers_and_meta(answers_path, meta_path, output_path=None):
    """
    Merges 'chatbot_answers.csv' with 'chatbot_meta_data.csv' so that each row
    in the final DataFrame has both the German- and English-related meta-data
    appended with the suffix '_de' or '_en'.
    
    Parameters
    ----------
    answers_path : str
        Path to the 'chatbot_answers.csv'
    meta_path : str
        Path to the 'chatbot_meta_data.csv'
    output_path : str, optional
        If given, saves the resulting DataFrame to this path as a CSV.
        
    Returns
    -------
    pd.DataFrame
        A DataFrame containing all columns from chatbot_answers plus two sets 
        of meta-data columns (one suffixed '_de' for German and one '_en' for English).
    """
    
    # --------------------------------------------------------------------
    # 1. Load both CSVs
    # --------------------------------------------------------------------
    df_answers = pd.read_csv(answers_path)
    df_meta = pd.read_csv(meta_path)
    
    # --------------------------------------------------------------------
    # 2. Prepare two copies of df_meta: one for German, one for English
    # --------------------------------------------------------------------
    # We'll rename chatbot_question -> chatbot_question_de/en
    # and all other columns -> columnname_de/en

    # --- 2.1 German copy ---
    df_meta_de = df_meta.copy()
    # rename 'chatbot_question' -> 'chatbot_question_de'
    df_meta_de.rename(columns={"chatbot_question": "chatbot_question_de"}, inplace=True)
    # rename other columns to have _de suffix
    for col in df_meta_de.columns:
        if col != "chatbot_question_de":
            df_meta_de.rename(columns={col: col + "_de"}, inplace=True)

    # --- 2.2 English copy ---
    df_meta_en = df_meta.copy()
    # rename 'chatbot_question' -> 'chatbot_question_en'
    df_meta_en.rename(columns={"chatbot_question": "chatbot_question_en"}, inplace=True)
    # rename other columns to have _en suffix
    for col in df_meta_en.columns:
        if col != "chatbot_question_en":
            df_meta_en.rename(columns={col: col + "_en"}, inplace=True)

    # --------------------------------------------------------------------
    # 3. Merge df_answers with the German meta copy
    #    on: df_answers.german_question_text_q == df_meta_de.chatbot_question_de
    # --------------------------------------------------------------------
    df_merged_german = pd.merge(
        df_answers,
        df_meta_de,
        how="left",
        left_on="german_question_text_q",
        right_on="chatbot_question_de"
    )
    
    # --------------------------------------------------------------------
    # 4. Merge the intermediate result with the English meta copy
    #    on: df_merged_german.english_question_text_q == df_meta_en.chatbot_question_en
    # --------------------------------------------------------------------
    df_merged_final = pd.merge(
        df_merged_german,
        df_meta_en,
        how="left",
        left_on="english_question_text_q",
        right_on="chatbot_question_en"
    )
    
    # --------------------------------------------------------------------
    # 5. (Optional) Reorder columns for clarity
    # --------------------------------------------------------------------
    desired_order = [
        # Original columns from df_answers
        "question_id_q",
        "german_question_text_q",
        "english_question_text_q",
        "chatbot_answer_de",
        "chatbot_answer_en",
        "chatbot_links_function_de",
        "chatbot_links_function_en",
        #"earliest_start_time",
        
        # German meta columns
        #"chatbot_question_de", == "german_question_text_q",
        "chatbot_context_de",
        "start_time_de",
        "end_time_de",
        "latency_de",
        "tool_run_used_de",
        "chatbot_query_de",
        "root_run_id_de",
        "tool_run_id_de",
        
        # English meta columns
        #"chatbot_question_en", == "english_question_text_q",
        "root_run_id_en",
        "start_time_en",
        "end_time_en",
        "latency_en",
        "chatbot_context_en",
        "chatbot_query_en",
        "tool_run_id_en",
        "tool_run_used_en"
    ]
    # Filter to keep only columns that actually exist:
    final_columns = [col for col in desired_order if col in df_merged_final.columns]
    df_merged_final = df_merged_final[final_columns]

    # --------------------------------------------------------------------
    # 6. (Optional) Save to CSV
    # --------------------------------------------------------------------
    if output_path:
        df_merged_final.to_csv(output_path, index=False, quoting=1)

    return df_merged_final


# Example usage:
merged_df = merge_chatbot_answers_and_meta(
    answers_path="../../data/chatbot_data/chatbot_answers.csv",
    meta_path="../../data/chatbot_data/chatbot_meta_data.csv",
    output_path="../../data/chatbot_data/chatbot_data_all.csv"
)
merged_df.head()


,question_id_q,german_question_text_q,english_question_text_q,chatbot_answer_de,chatbot_answer_en,chatbot_links_function_de,chatbot_links_function_en,chatbot_context_de,start_time_de,end_time_de,...,root_run_id_de,tool_run_id_de,root_run_id_en,start_time_en,end_time_en,latency_en,chatbot_context_en,chatbot_query_en,tool_run_id_en,tool_run_used_en
0,336,Wo erfahre ich etwas über Nachteilsausgleiche?,Where can I find information about compensatio...,Informationen über Nachteilsausgleiche an der ...,You can find information about compensations f...,https://www.uni-osnabrueck.de/studieninteressi...,https://www.uni-osnabrueck.de/studieninteressi...,Information taken from: https://www.uni-osnabr...,2025-01-09 13:39:44.881364,2025-01-09 13:40:01.737393,...,35ff9139-6805-4fce-a352-c29cfebfc813,2765d419-1ca7-48e5-90e2-e4c021a2d8e0,25689b6f-a50b-4b86-8cd6-3f162f3a7efd,2025-01-09 13:40:01.757631,2025-01-09 13:40:11.444568,9.686937,Information taken from: https://www.uni-osnabr...,Informationen über Ausgleichsmaßnahmen für Ben...,7adb08b7-ff45-4aa0-8503-32b5dd5bfc29,search tool used
1,9,Welche Master Studiengänge kann ich mit einem ...,Which master's degree programs can I do with a...,Mit einem Bachelor in Cognitive Science können...,I can only provide information related to the ...,https://www.uni-osnabrueck.de/studieninteressi...,NaN,Information taken from: https://www.uni-osnabr...,2025-01-09 13:40:11.470156,2025-01-09 13:40:26.124000,...,d6f72ebb-e6cf-4242-9fad-937c0c5ab1a2,fe9bd411-fcc4-4def-9853-e06008a59d35,d3b9c10b-f431-4332-823e-d0419d7dbebc,2025-01-09 13:40:26.152474,2025-01-09 13:40:27.820224,1.667750,NaN,NaN,NaN,no search tool used
2,203,Wie teuer ist ein Studium?,How expensive is a degree?,Die Kosten für ein Studium an der Universität ...,The cost of studying at the University of Osna...,https://www.uni-osnabrueck.de/studieninteressi...,https://www.uni-osnabrueck.de/studieninteressi...,Information taken from: https://www.uni-osnabr...,2025-01-09 13:40:27.836586,2025-01-09 13:40:54.795389,...,20c3bd6b-1208-46c6-8200-b1f0d39e2c98,bb021197-5996-43c4-9aab-d427f857b046,55510ed9-e31c-4dbe-9724-bfe231649410,2025-01-09 13:40:54.821807,2025-01-09 13:41:12.021437,17.199630,Information taken from: https://www.uni-osnabr...,Wie teuer ist ein Studium an der Universität O...,f943c191-1750-485c-9848-ce066da31703,search tool used
3,73,Sind die oberen Führungsebenen der Universität...,Are the upper management levels of the Univers...,Leider kann ich keine Informationen zu den dem...,"I'm sorry, but I cannot assist with questions ...",NaN,NaN,NaN,2025-01-09 13:41:12.039832,2025-01-09 13:41:13.762016,...,f4b9c2e2-4d46-4666-a65c-baec651bc583,NaN,67361f32-9652-4f88-9226-3d8405935954,2025-01-09 13:41:13.793434,2025-01-09 13:41:15.658327,1.864893,NaN,NaN,NaN,no search tool used
4,204,Welche Wohnmöglichkeiten werden für Studenten ...,What housing options are provided for students...,An der Universität Osnabrück stehen verschiede...,"At the University of Osnabrück, students have ...",https://www.uni-osnabrueck.de/studieninteressi...,https://www.uni-osnabrueck.de/studieninteressi...,Information taken from: https://www.uni-osnabr...,2025-01-09 13:41:15.680724,2025-01-09 13:41:39.388774,...,67e08bb2-63c4-4e99-bfee-15fab34a1944,ff1bdb63-e970-459c-83a7-4b51b4136033,e2923619-ff69-4eb8-99f3-89807085c198,2025-01-09 13:41:39.412349,2025-01-09 13:41:50.344009,10.931660,Information taken from: https://www.uni-osnabr...,Wohnmöglichkeiten für Studierende und Bewerbun...,a143d531-4646-4391-9d46-7b4b0142df14,search tool used


## testing

In [62]:
import pandas as pd
import re

# Load the CSV file
chatbot_data = pd.read_csv('../../data/chatbot_data/chatbot_data_all_TEST.csv')

# Function to extract links from 'chatbot_context_de'
def extract_links_from_context(context):
    # Extract all links after "Information taken from: "
    matches = re.findall(r'Information taken from: (https?://[^\s]+)', context)
    return matches  # Return raw list of links

# Function to compare links
def compare_links(row):
    # Extract links from 'chatbot_links_function_de'
    links_function = [link.strip() for link in str(row['chatbot_links_function_en']).split(';') if link.strip()]
    
    # Extract links from 'chatbot_context_de'
    links_context = extract_links_from_context(str(row['chatbot_context_en']))
    
    # Compare the two sets of links
    missing_from_function = set(links_context) - set(links_function)
    missing_from_context = set(links_function) - set(links_context)

    return pd.Series({
        'missing_from_function': list(missing_from_function),
        'missing_from_context': list(missing_from_context),
    })

# Apply the comparison to the DataFrame
comparison_results = chatbot_data.apply(compare_links, axis=1)

# Merge results back to the original data
chatbot_data = pd.concat([chatbot_data, comparison_results], axis=1)

# show only missing_from_context, missing_from_function columns
chatbot_data[['missing_from_function', 'missing_from_context']]


,missing_from_function,missing_from_context
0,[],[]
1,[],[nan]
2,[],[]
3,[],[nan]
4,[],[]
5,[],[nan]
6,[],[]
7,[],[]
8,[],[]
9,[],[]


In [63]:
chatbot_data['question_id_q'][2]

203

In [64]:
print(chatbot_data['chatbot_links_function_de'][2])

https://www.uni-osnabrueck.de/studieninteressierte/studienbeginn/kosten/;https://www.uni-osnabrueck.de/studieninteressierte/studiengaenge-a-z/;https://www.uni-osnabrueck.de/studium/organisatorisches/rueckmeldung/;https://www.uni-osnabrueck.de/universitaet/organisation/zentrale-verwaltung/gleichstellungsbuero/studierende/vereinbarkeit-studium-und-familie/


In [65]:
extract_links_from_context(chatbot_data['chatbot_context_de'][2])

['https://www.uni-osnabrueck.de/studieninteressierte/studienbeginn/kosten/',
 'https://www.uni-osnabrueck.de/studieninteressierte/studiengaenge-a-z/',
 'https://www.uni-osnabrueck.de/studium/organisatorisches/rueckmeldung/',
 'https://www.uni-osnabrueck.de/universitaet/organisation/zentrale-verwaltung/gleichstellungsbuero/studierende/vereinbarkeit-studium-und-familie/']

In [66]:
def extract_links(context):
        # Extract all links after "Information taken from:"
        matches = re.findall(r'Information taken from: (https?://[^\s]+)', context)
        if matches:
            # Format each link into an HTML list item
            links_html = ''.join(f'{link}' for link in matches)
            return f"<ul>{links_html}</ul>"
        return f"<p>{'BUUH'}</p>"

extract_links(chatbot_data['chatbot_context_de'][2])

'<ul>https://www.uni-osnabrueck.de/studieninteressierte/studienbeginn/kosten/https://www.uni-osnabrueck.de/studieninteressierte/studiengaenge-a-z/https://www.uni-osnabrueck.de/studium/organisatorisches/rueckmeldung/https://www.uni-osnabrueck.de/universitaet/organisation/zentrale-verwaltung/gleichstellungsbuero/studierende/vereinbarkeit-studium-und-familie/</ul>'

In [67]:
# what is the longest text in the chatbot_context_de column?
max_len = merged_df["chatbot_context_de"].str.len().max()

print(f"The longest text in the 'chatbot_context_de' column has {max_len} characters.")

import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

def calculate_api_call_cost(input_text: str, output_text: str, encoding: tiktoken, model_name: str) -> float:
    """
    Calculate the cost of an API call based on input and output texts.

    Args:
        input_text (str): The input text to be tokenized.
        output_text (str): The output text to be tokenized.
        model (str): The model name to determine the appropriate tokenizer.

    Returns:
        float: The total cost of the API call.
    """
    # Cast the input and output texts to strings
    input_text = str(input_text)
    output_text = str(output_text)
    
    input_tokens = encoding.encode(input_text)
    output_tokens = encoding.encode(output_text)

    # Calculate the number of tokens
    num_input_tokens = len(input_tokens)
   

    if num_input_tokens > 128000:
        print(f"Number of input tokens: {num_input_tokens} - Too many tokens for this model")

    return num_input_tokens
x = 0
y = 0
for context in merged_df["chatbot_context_de"]:
    cost = calculate_api_call_cost(context, "", encoding, "gpt-4o-mini")

    if cost > x:
        x = cost
        print(f"tokens for context: {cost}, in rw: {y}")
    y += 1

merged_df["english_question_text_q"][9]

The longest text in the 'chatbot_context_de' column has 103496.0 characters.
tokens for context: 8217, in rw: 0
tokens for context: 26004, in rw: 2
tokens for context: 36249, in rw: 9


'Do you need a NC for the German Studies teaching degree program?'

In [68]:
##########################DELETE THE BELOW CODE############################################
import pandas as pd

def reshape_and_merge_chatbot_datasets(
    answers_path: str,
    meta_path: str,
    output_path: str = None,
) -> pd.DataFrame:
    """
    Reshapes chatbot_answers.csv from wide format to long format (one row per question-language),
    then merges it with chatbot_meta_data.csv on the 'chatbot_question' column.
    
    :param answers_path: Path to 'chatbot_answers.csv'
    :param meta_path: Path to 'chatbot_meta_data.csv'
    :param output_path: Optional path to save the merged dataset as a CSV.
    :return: A pandas DataFrame containing the merged data.
    """

    # 1. Read both CSVs
    df_answers = pd.read_csv(answers_path)
    df_meta = pd.read_csv(meta_path)
    
    # 2. Reshape df_answers to a long format.
    #    We create two separate DataFrames: one for German and one for English, then concatenate.
    
    # --- German rows ---
    df_german = df_answers[
        [
            "question_id_q",
            "german_question_text_q",
            "chatbot_answer_de",
            "chatbot_links_function_de",
            "earliest_start_time",
        ]
    ].copy()
    df_german["language"] = "de"
    # Rename columns for a consistent long format
    df_german.rename(
        columns={
            "german_question_text_q": "chatbot_question",
            "chatbot_answer_de": "chatbot_answer",
            "chatbot_links_function_de": "chatbot_links_function",
        },
        inplace=True,
    )

    # --- English rows ---
    df_english = df_answers[
        [
            "question_id_q",
            "english_question_text_q",
            "chatbot_answer_en",
            "chatbot_links_function_en",
            "earliest_start_time",
        ]
    ].copy()
    df_english["language"] = "en"
    # Rename columns for a consistent long format
    df_english.rename(
        columns={
            "english_question_text_q": "chatbot_question",
            "chatbot_answer_en": "chatbot_answer",
            "chatbot_links_function_en": "chatbot_links_function",
        },
        inplace=True,
    )

    # Combine German + English rows into one long dataframe
    df_long = pd.concat([df_german, df_english], ignore_index=True)
    
    # 3. Merge the long dataframe with df_meta on chatbot_question
    #    Since each row in df_long has a 'chatbot_question' that should match df_meta's 'chatbot_question'.
    df_merged = pd.merge(
        df_long,
        df_meta,
        how="left",
        on="chatbot_question",
        suffixes=("_answers", "_meta")
    )
    
    # 4. (Optional) reorder columns for clarity
    desired_order = [
        "question_id_q",
        "language",
        "chatbot_question",
        "chatbot_answer",
        "chatbot_links_function",
        "chatbot_context",
        "chatbot_query",
        "earliest_start_time",
        "latency",
        "tool_run_used",
        "start_time",
        "end_time",
        "root_run_id",
        "tool_run_id",
    ]
    # Only keep columns that exist in df_merged
    df_merged = df_merged[[col for col in desired_order if col in df_merged.columns]]
    
    # 5. Save the result if an output path is provided
    if output_path:
        df_merged.to_csv(output_path, index=False)
    
    return df_merged


# Example usage:
merged_df = reshape_and_merge_chatbot_datasets(
    answers_path="../../data/chatbot_data/chatbot_answers.csv",
    meta_path="../../data/chatbot_data/chatbot_meta_data.csv",
    output_path="../../data/chatbot_data/chatbot_data_all.csv"
)
merged_df.head()


,question_id_q,language,chatbot_question,chatbot_answer,chatbot_links_function,chatbot_context,chatbot_query,earliest_start_time,latency,tool_run_used,start_time,end_time,root_run_id,tool_run_id
0,336,de,Wo erfahre ich etwas über Nachteilsausgleiche?,Informationen über Nachteilsausgleiche an der ...,https://www.uni-osnabrueck.de/studieninteressi...,Information taken from: https://www.uni-osnabr...,Nachteilsausgleiche Universität Osnabrück,2025-01-09 13:39:44.856609,16.856029,search tool used,2025-01-09 13:39:44.881364,2025-01-09 13:40:01.737393,35ff9139-6805-4fce-a352-c29cfebfc813,2765d419-1ca7-48e5-90e2-e4c021a2d8e0
1,9,de,Welche Master Studiengänge kann ich mit einem ...,Mit einem Bachelor in Cognitive Science können...,https://www.uni-osnabrueck.de/studieninteressi...,Information taken from: https://www.uni-osnabr...,Master Studiengänge nach einem Bachelor in Cog...,2025-01-09 13:39:44.856609,14.653844,search tool used,2025-01-09 13:40:11.470156,2025-01-09 13:40:26.124000,d6f72ebb-e6cf-4242-9fad-937c0c5ab1a2,fe9bd411-fcc4-4def-9853-e06008a59d35
2,203,de,Wie teuer ist ein Studium?,Die Kosten für ein Studium an der Universität ...,https://www.uni-osnabrueck.de/studieninteressi...,Information taken from: https://www.uni-osnabr...,Studiengebühren Universität Osnabrück,2025-01-09 13:39:44.856609,26.958803,search tool with summarization used,2025-01-09 13:40:27.836586,2025-01-09 13:40:54.795389,20c3bd6b-1208-46c6-8200-b1f0d39e2c98,bb021197-5996-43c4-9aab-d427f857b046
3,73,de,Sind die oberen Führungsebenen der Universität...,Leider kann ich keine Informationen zu den dem...,NaN,NaN,NaN,2025-01-09 13:39:44.856609,1.722184,no search tool used,2025-01-09 13:41:12.039832,2025-01-09 13:41:13.762016,f4b9c2e2-4d46-4666-a65c-baec651bc583,NaN
4,204,de,Welche Wohnmöglichkeiten werden für Studenten ...,An der Universität Osnabrück stehen verschiede...,https://www.uni-osnabrueck.de/studieninteressi...,Information taken from: https://www.uni-osnabr...,Wohnmöglichkeiten für Studenten und Bewerbung ...,2025-01-09 13:39:44.856609,23.708050,search tool used,2025-01-09 13:41:15.680724,2025-01-09 13:41:39.388774,67e08bb2-63c4-4e99-bfee-15fab34a1944,ff1bdb63-e970-459c-83a7-4b51b4136033
